In [1]:
import numpy as np

from astropy.io import fits
import astropy.units as u
#from astropy.units import cds
#cds.enable()


from astropy.io.ascii import masked
from astropy.table import Table
from astropy.timeseries import TimeSeries
import astropy.table as at

from astropy.timeseries import aggregate_downsample

import time
import os
from tqdm import tqdm

import kcorrect.response

In [2]:
SAVE_FILEPATH = 'C:/Users/paiaa/Documents/Research/Blanton Lab/WISE variations/'
IMPORT_FILEPATH = 'C:/Users/paiaa/Documents/Research/Blanton Lab/WISE variations/'

In [3]:
def import_manga():
    manga_file = fits.open(IMPORT_FILEPATH + "mnsa-0.3.0.fits")
    hdu_manga = manga_file[2]
    manga_data = hdu_manga.data
    return manga_data

def write_query_table():
    manga_file = fits.open(IMPORT_FILEPATH + "mnsa-0.3.0.fits")
    RA = manga_file[1].data['objra']
    dec = manga_file[1].data['objdec']
    
    table = Table([RA, dec], names=('ra', 'dec'))
    table.write('objects_all.tbl', format = 'ipac', overwrite = True)
    return table

In [4]:
def filter_data(ts):
    ts = ts[ts['qual_frame'] > 0]
    ts = ts[ts['cc_flags'] == '0000']
    ts = ts[ts['sigra'] < 1]
    ts = ts[ts['sigdec'] < 1]
    
    mask = ((ts['w1mpro'] == 'null') | (ts['w2mpro'] == 'null') | (ts['w1sigmpro'] == 'null') | (ts['w2sigmpro'] == 'null'))
    ts = ts[~mask]
    return ts

def change_dtype(ts):
    ts['w2mpro'] = ts['w2mpro'].astype(float)
    ts['w1mpro'] = ts['w1mpro'].astype(float)
    ts['w2sigmpro'] = ts['w2sigmpro'].astype(float)
    ts['w1sigmpro'] = ts['w1sigmpro'].astype(float)
    return ts

def vega2ab(ts):
    rd = kcorrect.response.ResponseDict()
    rd.load_response('wise_w2')
    ts['w2mpro'] = ts['w2mpro'] + rd['wise_w2'].vega2ab
    
    rd.load_response('wise_w1')
    ts['w1mpro'] = ts['w1mpro'] + rd['wise_w1'].vega2ab
    
    return ts
    
def bin_data(ts, freq, index):
    ts = ts[ts['cntr_01'] == index]
    binned_ts = aggregate_downsample(ts, time_bin_size = freq * u.day, n_bins=26, aggregate_func=np.size)
    binned_ts_avg = aggregate_downsample(ts, time_bin_size = freq * u.day, n_bins=26, aggregate_func=np.nanmean)
    binned_ts_var = aggregate_downsample(ts, time_bin_size = freq * u.day, n_bins=26, aggregate_func=np.nanvar)
    
    return binned_ts, binned_ts_avg, binned_ts_var, ts

In [5]:
def process_data(ts):
    ts = filter_data(ts)
    ts.keep_columns(['time', 'cntr_01', 'w2mpro', 'w2sigmpro', 'w1mpro', 'w1sigmpro'])
    ts.sort(['cntr_01', 'time'])
    ts = change_dtype(ts)
    ts = vega2ab(ts)
    
    return ts

In [6]:
neowise_data = TimeSeries.read(IMPORT_FILEPATH + 'neowise_2arcsec_catalog_search_results_all.csv', format='csv', time_column='mjd', time_format='mjd')
allsky_data = TimeSeries.read(IMPORT_FILEPATH + 'allsky_2arcsec_catalog_search_results_all_qualframe.csv', format='csv', time_column='mjd', time_format='mjd')
wise_data = at.vstack([neowise_data, allsky_data])



# Statistical Calculations

In [7]:
def mean_mag_per_epoch(ts, binned_ts, mag): 
    obs_num = binned_ts['cntr_01'].data.data
    index = 0
    arr = np.array([])

    for i in range(obs_num.shape[0]):

        temp = ts[mag].data[index:index+obs_num[i]]
        mean = np.nansum(temp)/temp[np.isfinite(temp)].shape[0]
        arr = np.append(arr, mean)
        
        index += obs_num[i]
    return arr

##FIX THIS (fixed now)
def std_err_per_epoch(ts, binned_ts, mag, err): 
    obs_num = binned_ts['cntr_01'].data.data
    index = 0
    arr = np.array([])

    for i in range(obs_num.shape[0]):
        
        temp_mag = ts[mag].data[index:index+obs_num[i]]
        temp_err = ts[err].data[index:index+obs_num[i]]
        
        if (temp_mag[np.isfinite(temp_mag)]).shape[0] != 1:
            std_err = np.nanstd(temp_mag, ddof=1)#/np.sqrt(temp_mag[np.isfinite(temp_mag)].shape[0]-1)

            arr = np.append(arr, std_err)
            
        else:
            #print('w2mpro:', temp_mag)
            #print('w2sigmpro:', temp_err, 'nonzero indices:', np.nonzero(temp_err))
            std_err = temp_err[np.nonzero(temp_err)[0]]
            arr = np.append(arr, std_err)
        
        index += obs_num[i]
    #print('std_err:', arr)
    return arr


def expected_var_per_epoch_sigma(ts, binned_ts, err): 
    obs_num = binned_ts['cntr_01'].data.data
    index = 0
    arr = np.array([])
    for i in range(obs_num.shape[0]):

        temp = ts[err].data[index:index+obs_num[i]]
        var = np.nansum(np.square(temp))/(temp[np.isfinite(temp)].shape[0])
        arr = np.append(arr, var)
        
        index += obs_num[i]
    return arr


def expected_var_per_epoch_mags(ts, binned_ts, mag): 
    obs_num = binned_ts['cntr_01'].data.data
    index = 0
    arr = np.array([])
    for i in range(obs_num.shape[0]):
        
        temp = ts[mag].data[index:index+obs_num[i]]
        var = np.nanvar(temp, ddof=1)
        arr = np.append(arr, var)
        
        index += obs_num[i]
    #print('var:', arr)
    return arr

def expected_var_per_epoch_mags_mean(ts, binned_ts, mag): 
    obs_num = binned_ts['cntr_01'].data.data
    index = 0
    arr = np.array([])
    for i in range(obs_num.shape[0]):

        temp = ts[mag].data[index:index+obs_num[i]]
        var = np.nanvar(temp, ddof=1)/(temp[np.isfinite(temp)].shape[0])
        arr = np.append(arr, var)
        
        index += obs_num[i]
    return arr

def expected_var_per_epoch_sigma_mean(ts, binned_ts, err): 
    obs_num = binned_ts['cntr_01'].data.data
    index = 0
    arr = np.array([])
    for i in range(obs_num.shape[0]):

        temp = ts[err].data[index:index+obs_num[i]]
        var = np.nansum(np.square(temp))/np.square(temp[np.isfinite(temp)].shape[0])
        arr = np.append(arr, var)
        
        index += obs_num[i]
    return arr

In [8]:
def obs_var_all(ts, binned_ts_avg, mag):
    
    var = np.nanvar(binned_ts_avg[mag], ddof=1)
    
    return var

def expected_var_all_sigma(ts, binned_ts, err):
    per_epoch_var = expected_var_per_epoch_sigma_mean(ts, binned_ts, err)
    
    var = np.nanmean(per_epoch_var)
    
    return var

def expected_var_all_mags(ts, binned_ts, mag):
    per_epoch_var = expected_var_per_epoch_mags_mean(ts, binned_ts, mag)
    
    var = np.nanmean(per_epoch_var)
    
    return var

In [9]:
def plateifu():
    
    return MANGA_DATA['plateifu']

def date_per_epoch(ts, binned_ts):
    
    return binned_ts['time_bin_start'].value

def obs_per_epoch(ts, binned_ts):
    
    return binned_ts['cntr_01'].data.data

def epochs_per_obj(ts, binned_ts):
    
    return np.nonzero(binned_ts['cntr_01'])[0].shape[0]

# Creating Catalog

In [10]:
def calculate_stats(ts_full):
    skip_counter = 0
    pifu = plateifu()
    index = ts_full['cntr_01'].max()+1

    variability_cat = Table(names = np.array(['plateifu', 'mjd', 'obs_per_ep', 'epoch_count', 'mean_W1_per_epoch', 'mean_W2_per_epoch', 'err_W1_per_epoch', 'err_W2_per_epoch', 
                        'expected_W1_var_errs', 'expected_W2_var_errs', 'expected_W1_var_mags', 'expected_W2_var_mags', 'expected_W1_var_mean_errs', 'expected_W2_var_mean_errs', 'expected_W1_var_mean_mags', 'expected_W2_var_mean_mags', 'observed_W1_var', 'observed_W2_var',
                     'expected_W1_var_all_errs', 'expected_W2_var_all_errs','expected_W1_var_all_mags' ,'expected_W2_var_all_mags']),
                        dtype = np.array(['<U12', '26f8', '26i8', 'i8', '26f8', '26f8', '26f8','26f8', '26f8', '26f8','26f8', '26f8', '26f8', 
                                '26f8', '26f8', '26f8', 'f8', 'f8', 'f8', 'f8', 'f8', 'f8']))
    
    for i in tqdm(range(1, index)):
        #print(i)
        ts = ts_full[ts_full['cntr_01'] == i]
        
        if ts['cntr_01'].size <2:
            epochs = 26
            w2a, w1a = np.ones(epochs)*-9999., np.ones(epochs)*-9999.
            w2b, w1b = np.ones(epochs)*-9999., np.ones(epochs)*-9999.
            w2c, w1c = np.ones(epochs)*-9999., np.ones(epochs)*-9999.
            w2d, w1d = np.ones(epochs)*-9999., np.ones(epochs)*-9999.
            w2h, w1h = np.ones(epochs)*-9999., np.ones(epochs)*-9999.
            w2i, w1i = np.ones(epochs)*-9999., np.ones(epochs)*-9999.
            w2e, w1e = -9999., -9999.
            w2f, w1f = -9999., -9999.
            w2g, w1g = -9999., -9999.
            
            epoch_date = np.ones(epochs)*-9999.
            obs_per_ep = (np.ones(epochs)*-9999).astype(int)
            ep_per_obj = int(-9999)
            
            variability_cat.add_row([pifu[i-1], epoch_date, obs_per_ep, ep_per_obj, w1a, w2a, w1b, w2b, w1c, w2c, w1d, w2d, w1i, w2i,
                     w1h, w2h, w1e, w2e, w1f, w2f, w1g, w2g])
            
            skip_counter += 1
            
            #print('skipped')
            continue
        binned_ts, binned_ts_avg, binned_ts_var, ts = bin_data(ts, 181, int(i)) #changed freq=170 for 5.2.0
        
        w2a = mean_mag_per_epoch(ts, binned_ts, 'w2mpro')
        w1a = mean_mag_per_epoch(ts, binned_ts, 'w1mpro')
        
        w2b = std_err_per_epoch(ts, binned_ts, 'w2mpro', 'w2sigmpro')
        w1b = std_err_per_epoch(ts, binned_ts, 'w1mpro', 'w1sigmpro')
        
        w2c = expected_var_per_epoch_sigma(ts, binned_ts, 'w2sigmpro')
        w1c = expected_var_per_epoch_sigma(ts, binned_ts, 'w1sigmpro')
        
        w2d = expected_var_per_epoch_mags(ts, binned_ts, 'w2mpro')
        w1d = expected_var_per_epoch_mags(ts, binned_ts, 'w1mpro')
        
        w2i = expected_var_per_epoch_sigma_mean(ts, binned_ts, 'w2sigmpro')
        w1i = expected_var_per_epoch_sigma_mean(ts, binned_ts, 'w1sigmpro')
        
        w2h = expected_var_per_epoch_mags_mean(ts, binned_ts, 'w2mpro')
        w1h = expected_var_per_epoch_mags_mean(ts, binned_ts, 'w1mpro')
        
        w2e = obs_var_all(ts, binned_ts_avg, 'w2mpro')
        w1e = obs_var_all(ts, binned_ts_avg, 'w1mpro')
        
        w2f = expected_var_all_sigma(ts, binned_ts, 'w2sigmpro')
        w1f = expected_var_all_sigma(ts, binned_ts, 'w1sigmpro')
        
        w2g = expected_var_all_mags(ts, binned_ts, 'w2mpro')
        w1g = expected_var_all_mags(ts, binned_ts, 'w1mpro')
        

        epoch_date = date_per_epoch(ts, binned_ts)
        obs_per_ep = obs_per_epoch(ts, binned_ts)
        ep_per_obj = epochs_per_obj(ts, binned_ts)
        
        variability_cat.add_row([pifu[i-1], epoch_date, obs_per_ep, ep_per_obj, w1a, w2a, w1b, w2b, w1c, w2c, w1d, w2d, w1i, w2i, 
                     w1h, w2h, w1e, w2e, w1f, w2f, w1g, w2g])
        
    print(skip_counter, ' objects have less than 2 observations')
    print('**************************')
    print('adding coordinates')
    ra, dec = get_coords(variability_cat)
    variability_cat.add_column(ra, name='ra', index=1)
    variability_cat.add_column(dec, name='dec', index=2)
    print('**************************')
    print('creating flags')
    viflags, epflags, varflags = create_flags(variability_cat)
    variability_cat.add_column(viflags.astype(int), name='vi_flag')
    variability_cat.add_column(epflags.astype(int), name='epoch_flag')
    variability_cat.add_column(varflags.astype(int), name='var_flag')
    print('**************************')

    return variability_cat

def create_flags(table):
    
    visual_inspection_flag = np.array([])
    epoch_flag = np.array([])
    var_flag = np.array([])
    
    for pifu in table['plateifu']:
        if pifu in ['8132-6102', '8145-6102', '8239-3701', '8333-3703', 
                    '8445-3701', '8447-9102','8483-12701', '8616-12701', '9024-12705',
                    '10217-3704', '10515-6102', '12078-6102', '12085-1902', '12651-3703', '12667-6104']:
                   #'11863-3703', '9046-1902', '8133-12702', '9497-12705']: #this row doesn't have manga w1/w2
            visual_inspection_flag = np.append(visual_inspection_flag, 1)
        else:
            visual_inspection_flag = np.append(visual_inspection_flag, 0)
    
    for pifu in table['plateifu']:
        if pifu in table[table['epoch_count']<4]['plateifu']:
            epoch_flag = np.append(epoch_flag, 1)
        else:
            epoch_flag = np.append(epoch_flag, 0)
            
    catalog = Table.read(IMPORT_FILEPATH+'/Final Plots/' + 'variability_catalog', format='ascii')
    
    for pifu in table['plateifu']:
        if pifu in catalog['Plateifu']:
            var_flag = np.append(var_flag, 1)
        else:
            var_flag = np.append(var_flag, 0)
            
    return visual_inspection_flag, epoch_flag, var_flag

def get_coords(table):
    manga_file = fits.open(IMPORT_FILEPATH + "mnsa-0.3.0.fits")
    mf = manga_file[1].data
    
    ra = np.array([])
    dec = np.array([])
    
    for pifu in table['plateifu']:
        ra = np.append(ra, mf[mf['plateifu'] == pifu]['objra'])
        dec = np.append(dec, mf[mf['plateifu'] == pifu]['objdec'])
    
    return ra, dec

# Main

In [11]:
def main(wise_data, version='temp'):
    
    t1 = time.time()
    try:
        print('processing data')
        w_data = process_data(wise_data)
    except:
        print('data processing failed!')
        return None, None
    
    try: 
        print('performing statistical calculations')
        catalog = calculate_stats(w_data)
    except:
        print('statistical calculations failed!')
        return w_data, None
    
    try:
        print('saving catalog')
        catalog.write(SAVE_FILEPATH + 'manga-wise-variable-'+version+'.fits', overwrite=True)
    except:
        print('unable to save the catalog!')
        return catalog
    

    t2 = time.time()
    
    print('**************************')
    print('total time taken:', np.round(t2-t1, 2), 'seconds')
    return w_data, catalog
    

In [13]:
if __name__ == "__main__":
    try:
        MANGA_DATA = import_manga()
    except:
        print('unable to import MaNGA data!')
    ts1, cat1 = main(wise_data, version='0.3.1')

processing data
